In [ ]:
import typing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import QuantLib as ql
import yfinance as yf
import pandas_market_calendars as mcal
import datetime
import time
from pathlib import Path
import os

# Self-written modules
from yq.scripts import models
from yq.scripts import heston_func
from yq.utils import option
from yq.utils import calendar
from yq.scripts import simulation as sm
from sc import constants as cs
from sy.variance_reduction import apply_control_variates
from sy.interest_rate import populate_bond_table, get_period
from sy.calibration import apply_empirical_martingale_correction

In [ ]:
print(cs.ASSET_NAMES)

# Fetch stock data

In [ ]:
historical_start_date = '2022-08-09'
# Define the ticker list
ticker_list = ['LONN.SW', 'SIKA.SW']

# Fetch the data
data = yf.download(ticker_list, historical_start_date)['Adj Close'] # Auto adjust is false
display(data.tail(20))
data.plot()
display(data.loc['2023-11-07']['LONN.SW'])

plt.figure(figsize=(3,2))
sns.heatmap(data.corr(), cmap="Reds", annot=True)
plt.show()

In [ ]:
params = {
    'data': data,
    'ticker_list': ['LONN.SW', 'SIKA.SW']
}
trading_calendar = calendar.SIXTradingCalendar()
gbm = models.PricingModel(params = params)

In [ ]:
# Copy the start_time_str from the folders
# product_est_date_sim_data_df_list = sm.read_sim_data(
#         model_name='gbm',
#         start_time_str='20231110_233950_693680', 
#         prod_est_start_date=pd.Timestamp('2023-08-09'), 
#         prod_est_end_date=pd.Timestamp('2023-08-10'))
# print(type(product_est_date_sim_data_df_list)[0])
# n_sim_on_day = pd.concat(product_est_date_sim_data_df_list[0], axis=1)
# ax = n_sim_on_day.plot(alpha=0.6, legend=False)
# ax.figure.show()


# Testing diff h

In [ ]:
bus_date_range = trading_calendar.create_six_trading_dates('2023-08-09', '2023-08-09')
# display(bus_date_range)
# print(bus_date_range.index.to_list())
start_time_acc = datetime.datetime.now() # Track the nth attempt
print(start_time_acc)
n_sim = 1
for product_est_date in bus_date_range.index:
    try:
        start_time = time.time()

        sim_data_df = []
        for sim in range(n_sim):
            sim_start_date = trading_calendar.add_trading_day(product_est_date, 1)
            
            sim_data = gbm.multi_asset_gbm(
                sim_start_date=sim_start_date, 
                hist_window=252, 
                sim_window=trading_calendar.calculate_business_days(sim_start_date, cs.FINAL_FIXING_DATE), 
                h_adjustment=[0, 0])

            sim_data_h = gbm.multi_asset_gbm(
                sim_start_date=sim_start_date, 
                hist_window=252, 
                sim_window=trading_calendar.calculate_business_days(sim_start_date, cs.FINAL_FIXING_DATE), 
                h_adjustment=[1000, 0])
        end_time = time.time()
        elapsed_time = end_time - start_time
        min, sec = divmod(elapsed_time, 60)
        print(f"The elapsed time is for {n_sim} is {int(min)} minutes, {int(sec)} seconds")
        
        S_T_1 = sim_data.loc[cs.FINAL_FIXING_DATE, 'LONN.SW']
        S_T_2 = sim_data_h.loc[cs.FINAL_FIXING_DATE,'LONN.SW']
        print(S_T_1 / S_T_2)
        
    except Exception as e:
        # Log the error with the date that caused it
        raise Exception("MultiGBM has error.")


In [ ]:
print(489/1489)


In [ ]:
params = {
    'data': data,
    'ticker_list': ['LONN.SW', 'SIKA.SW']
}
trading_calendar = calendar.SIXTradingCalendar()
heston = models.PricingModel(params = params)

In [ ]:
bus_date_range = trading_calendar.create_six_trading_dates('2023-08-09', '2023-08-09')
# display(bus_date_range)
# print(bus_date_range.index.to_list())
start_time_acc = datetime.datetime.now() # Track the nth attempt
print(start_time_acc)
n_sim = 1
for product_est_date in bus_date_range.index:
    try:
        start_time = time.time()

        sim_data_df = []
        for sim in range(n_sim):
            sim_start_date = trading_calendar.add_trading_day(product_est_date, 1)
            
            sim_data = heston.multi_asset_heston_model(
                sim_start_date=sim_start_date, 
                hist_window=252, 
                sim_window=trading_calendar.calculate_business_days(sim_start_date, cs.FINAL_FIXING_DATE), 
                h_adjustment=[0, 0])

            sim_data_h = heston.multi_asset_heston_model(
                sim_start_date=sim_start_date, 
                hist_window=252, 
                sim_window=trading_calendar.calculate_business_days(sim_start_date, cs.FINAL_FIXING_DATE), 
                h_adjustment=[1, 0])
        end_time = time.time()
        elapsed_time = end_time - start_time
        min, sec = divmod(elapsed_time, 60)
        print(f"The elapsed time is for {n_sim} is {int(min)} minutes, {int(sec)} seconds")
        
        S_T_1 = sim_data.loc[cs.FINAL_FIXING_DATE, 'LONN.SW']
        S_T_2 = sim_data_h.loc[cs.FINAL_FIXING_DATE,'LONN.SW']
        print(S_T_1 / S_T_2)
        
    except Exception as e:
        # Log the error with the date that caused it
        raise Exception("MultiGBM has error.")


In [ ]:
489/490


In [ ]:
display(sim_data)
display(sim_data_h)

In [ ]:
print(355/1404.907)

In [ ]:
0.32840832773673606

# Normal sims

In [ ]:
bus_date_range = trading_calendar.create_six_trading_dates('2023-08-09', '2023-08-09')
# display(bus_date_range)
# print(bus_date_range.index.to_list())
start_time_acc = datetime.datetime.now() # Track the nth attempt
print(start_time_acc)
n_sim = 1
for product_est_date in bus_date_range.index:
    try:
        start_time = time.time()

        sim_data_df = []
        for sim in range(n_sim):
            sim_start_date = trading_calendar.add_trading_day(product_est_date, 1)
            
            sim_data = gbm.multi_asset_gbm(
                sim_start_date=sim_start_date, 
                hist_window=252, 
                sim_window=trading_calendar.calculate_business_days(sim_start_date, cs.FINAL_FIXING_DATE), 
                h_adjustment=[0, 0])
            sim_data_df.append(sim_data)
            
            sm.store_sim_data(start_time_acc=start_time_acc,
                           model_name='gbm',
                           sim_data=sim_data,
                           product_est_date=product_est_date,
                           sim=sim)

        end_time = time.time()
        elapsed_time = end_time - start_time
        min, sec = divmod(elapsed_time, 60)
        print(f"The elapsed time is for {n_sim} is {int(min)} minutes, {int(sec)} seconds")
        
    except Exception as e:
        # Log the error with the date that caused it
        raise Exception("MultiGBM has error.")
    
    
    # TODO: Tag the sim_data with the dates
    # TODO: Add the concate the yfinance data from initial fixing date to the sim_data
    # TODO: Break the df into 2 df, each with date as one of the columns (not the index) for sc payoff function
    # Get the data?

    
    # TODO: Store the payoff df to the class and plot?

# TODO: Implement the second simulation with the control variate method? Another loop or same loop?

# sim_data.columns = ["simulated_" + ticker for ticker in ticker_list]

# A dataframe with both validation data and simulated data


In [ ]:
sim_data_df

In [ ]:
# generate bond table
path = '../data/bond'

bond_yield = None
for file in os.listdir(path):
   df = pd.read_csv(os.path.join(path, file))[['Date','Price']]
   df.rename(columns={'Price':file.split(' ')[1]}, inplace=True)
   df['Date'] = pd.to_datetime(df['Date'],format='%m/%d/%Y')
   df = df.set_index('Date').iloc[::-1]
   if bond_yield is None:
      bond_yield = df
   else:
      bond_yield = pd.concat([bond_yield, df], axis=1)
bond_yield = bond_yield.interpolate()
bond_yield = bond_yield.reindex(sorted(bond_yield.columns, key=lambda x: get_period(x)), axis=1) # sort index
bond_price = pd.DataFrame(index=bond_yield.index)
for col in bond_yield.columns:
    bond_price[col] = bond_yield[col].apply(lambda x: np.exp(-x/100*get_period(col)))
bond_price

In [ ]:
sim_data = gbm.adjust_interest_rate(bond_price, sim_data_df[0])
sim_data

In [ ]:
# Variance reduction
today = pd.Timestamp('2023-08-09')
r = 0.0167
# calculate means
s_01  = 1
s_02 = 1.2
mu1 = s_01 * np.exp(r*(cs.FINAL_FIXING_DATE - today).days/252)
mu2 = s_02 * np.exp(r*(cs.FINAL_FIXING_DATE  - today).days/252)
print(mu1, mu2)


In [ ]:
# TODO pass me ST for lonn and sika as well as payoffs
# payoffs_cv = apply_control_variates(__place_holder__, __place_holder__, mu1, mu2, payoffs)